In [1]:
import os
CWD = os.getcwd()
import numpy as np
import pandas as pd 
import shutil
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import image
from tensorflow.image import ResizeMethod
import derm_ita as dita
import h5py

2024-02-11 19:03:43.825408: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 19:03:43.825453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 19:03:43.826341: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 19:03:43.832369: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
metadata_ISIC = pd.read_excel(f'{CWD}/metadata_ISIC.xlsx', index_col='index')
num_imgs = len(metadata_ISIC['id'])
print(metadata_ISIC)

                    id      label  malignance  skin_tone
index                                                   
0      ISIC_000000.jpg     benign           0         12
1      ISIC_000001.jpg     benign           0         12
2      ISIC_000002.jpg  malignant           1         12
3      ISIC_000003.jpg     benign           0         12
4      ISIC_000004.jpg  malignant           1         56
...                ...        ...         ...        ...
1971   ISIC_015251.jpg  malignant           1         12
1972   ISIC_015256.jpg  malignant           1         34
1973   ISIC_015260.jpg     benign           0         12
1974   ISIC_015284.jpg  malignant           1         12
1975   ISIC_015295.jpg     benign           0         56

[1976 rows x 4 columns]


In [3]:
def ITA_FST(image):
    img_ita = dita.get_cropped_center_ita(image)
    groh_FST = None
    # Following the standard set in DDI.
    if 23 <= img_ita:
        groh_FST = 12
    elif 0 <= img_ita < 23:
        groh_FST = 34
    elif img_ita < 0:
        groh_FST = 56

    return groh_FST

In [4]:
# skin_tones = np.empty(shape=[num_imgs, 1], dtype=np.uint8)
# images = np.empty(shape=[num_imgs, 64, 64, 3], dtype=np.uint8)
# for i in range(num_imgs):
#     id = metadata_ISIC['id'][i]
#     image = tf.keras.utils.load_img(f'{CWD}/all_images_original/{id}', target_size = (64, 64))
#     skin_tones[i] = ITA_FST(image)
#     image = tf.keras.utils.img_to_array(image, dtype=np.uint8)
#     # assert image.shape == (64, 64, 3)
#     # assert image.dtype == np.uint8
#     images[i] = image
#     tf.keras.utils.save_img(f'{CWD}/all_images/{id}', image)

# metadata_ISIC['skin_tone'] = skin_tones 

In [5]:
print(metadata_ISIC)
metadata_ISIC.to_excel(f'{CWD}/metadata_ISIC.xlsx')

                    id      label  malignance  skin_tone
index                                                   
0      ISIC_000000.jpg     benign           0         12
1      ISIC_000001.jpg     benign           0         12
2      ISIC_000002.jpg  malignant           1         12
3      ISIC_000003.jpg     benign           0         12
4      ISIC_000004.jpg  malignant           1         56
...                ...        ...         ...        ...
1971   ISIC_015251.jpg  malignant           1         12
1972   ISIC_015256.jpg  malignant           1         34
1973   ISIC_015260.jpg     benign           0         12
1974   ISIC_015284.jpg  malignant           1         12
1975   ISIC_015295.jpg     benign           0         56

[1976 rows x 4 columns]


In [6]:
images = np.empty(shape=[num_imgs, 64, 64, 3], dtype=np.uint8)
for i in range(num_imgs):
    id = metadata_ISIC['id'][i]
    image = tf.keras.utils.load_img(f'{CWD}/all_images/{id}', target_size = (64, 64))
    images[i] = image

In [7]:
with h5py.File(f'{CWD}/ISIC.h5','w') as f:
    f.create_dataset('images', data = images)
    f.create_dataset('labels', data = np.array(metadata_ISIC['malignance'], dtype=np.uint8).reshape(num_imgs,1))

In [8]:
metadata_ISIC = pd.read_excel(f'{CWD}/metadata_ISIC.xlsx', index_col = 'index')
print(metadata_ISIC)

                    id      label  malignance  skin_tone
index                                                   
0      ISIC_000000.jpg     benign           0         12
1      ISIC_000001.jpg     benign           0         12
2      ISIC_000002.jpg  malignant           1         12
3      ISIC_000003.jpg     benign           0         12
4      ISIC_000004.jpg  malignant           1         56
...                ...        ...         ...        ...
1971   ISIC_015251.jpg  malignant           1         12
1972   ISIC_015256.jpg  malignant           1         34
1973   ISIC_015260.jpg     benign           0         12
1974   ISIC_015284.jpg  malignant           1         12
1975   ISIC_015295.jpg     benign           0         56

[1976 rows x 4 columns]
